# Importando as bibliotecas necessárias

In [ ]:
import numpy as np
import pandas as pd
from keras.api.layers import LSTM, Dense
from keras.api.models import Sequential
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Carregamento dos Dados Pré-processados

In [ ]:
processed_data_path = '../data/processed/normalized_stock_data.csv'
stock_data = pd.read_csv(processed_data_path, index_col=0).sort_values('Date')

stock_data

# Divisão dos Dados em Treino e Teste

In [ ]:
train_size = int(len(stock_data) * 0.8)
train_data, test_data = stock_data[:train_size], stock_data[train_size:]

# Preparação dos Dados para o Modelo LSTM

In [8]:
def create_sequences(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i : (i + time_step), :])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)


time_step = 60

In [ ]:
cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Preparando os dados de treino
train_data_values = train_data[cols].to_numpy()
X_train, y_train = create_sequences(train_data_values, time_step)

# Preparando os dados de teste
test_data_values = test_data[cols].to_numpy()
X_test, y_test = create_sequences(test_data_values, time_step)

# Construção do Modelo LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 6)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compilando o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Resumo do modelo
model.summary()

c:\Users\miche\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 50)         │        11,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,901 (128.52 KB)

 Trainable params: 32,901 (128.52 KB)

 Non-trainable params: 0 (0.00 B)

# Treinamento do Modelo

In [11]:
model.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 493s 38ms/step - loss: 2.5331e-04
Epoch 2/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 512s 41ms/step - loss: 4.4433e-05
Epoch 3/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 473s 38ms/step - loss: 3.0129e-05
Epoch 4/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 458s 36ms/step - loss: 1.9033e-05
Epoch 5/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 448s 36ms/step - loss: 1.8891e-05
Epoch 6/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 451s 36ms/step - loss: 1.7931e-05
Epoch 7/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 444s 35ms/step - loss: 1.7384e-05
Epoch 8/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 456s 36ms/step - loss: 1.1518e-05
Epoch 9/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 455s 36ms/step - loss: 1.2186e-05
Epoch 10/10
12590/12590 ━━━━━━━━━━━━━━━━━━━━ 448s 36ms/step - loss: 1.1041e-05


# Avaliação do Modelo

In [12]:
# Fazendo previsões com o modelo treinado
predictions = model.predict(X_test)

97/97 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(stock_data[['Close']])
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

In [14]:
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')

MAE: 0.010510147977992866
RMSE: 0.011915325856781981
MAPE: 1.6991471865303702%


# Salvando o modelo

In [15]:
model_save_path = '../saved_models/lstm_stock_prediction_model.h5'
model.save(model_save_path)

# Vizualizando as previsões